In [1]:
%matplotlib notebook 
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

ModuleNotFoundError: No module named 'matplotlib'

# 1. 오토인코더 모델 설계

In [2]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 12),
            nn.Tanh(),
            nn.Linear(12, 3)   # 잠재공간 latent space (3*3), Compressed representation
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.Tanh(),
            nn.Linear(12, 64),
            nn.Tanh(),
            nn.Linear(64, 128),
            nn.Tanh(),
            nn.Linear(128, 28*28),
            nn.Tanh()       # Output a value between 0 and 1
        )

# 순전파 : 748차원 -> 3차원으로 
    def forward(self, x): 
        x_encoded = self.encoder(x) # X:input 데이터 784차원, x_enoded = (3,3) 잠재공간 
        x = self.decoder(x_encoded) # 2. (3, 3)으로 축소된 데이터를 디코더에 넣어서 다시 x를 얻음음
        return x, x_encoded

# 2. Train, Test 데이터 준비

In [3]:
# 데이터 로드 및 변환 : 데이터를 tensor형태로 변환
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
# 1) train데이터 변환 후 -> 로드 
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # 배치 사이즈 64, 랜덤하게 
# 학습 시, 미니배치 경사하강법을 사용하겠다! => 64개씩 데이터를 랜덤하게 모아서 학습하겠다

# 2) test 데이터 변환 후 -> 로드 
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=True)  # Larger batch size for testing

# 3. 모델 준비비

In [4]:
# 모델 초기화 
model = Autoencoder() # 모델 생성성
criterion = nn.MSELoss() # 손실함수 mse 
optimizer = optim.Adam(model.parameters(), lr=1e-3) # 옵티마이저 = adam 

# 4. Train 설정

In [ ]:
# Training loop
num_epochs = 10 # 학습 10번 돌리기
for epoch in range(num_epochs):
    for data in train_loader:  # 변환 후 배치 된 train_loader 를 불러서
        img, _ = data
        img = img.view(img.size(0), -1)  # Flatten the images
        output,_ = model(img) # 784->3->784 차원으로 output 결과
        loss = criterion(output, img) # MSE로 output, img(input)사이의 손실계산산

        optimizer.zero_grad()
        loss.backward() # 손실을 사용하여 역전파 구하기
        optimizer.step() # 옵티마이저(경사하강법법)로 각각 가중치 업데이트

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training Complete")

#5.  test 단계계

In [6]:

# 테스트 데이터 준비비
dataiter = iter(test_loader)
images, _ = next(dataiter)
images_flattened = images.view(images.size(0), -1) # flatten : # 미니배치기에 (1000, 784(28*28))-> 784차원원

# test 이미지를 순전파에 넣기
output, _ = model(images_flattened) # test 데이터를 모델에 넣고 output : 784차원원
# 실제 눈으로 보기 위해 output인 flatten된 결과값을 2차원의 28*28로 변환환
output = output.view(output.size(0), 1, 28, 28).detach() 

# 결과 비교 : output의 결과가 input과 얼마나 비슷한지 시각화해서 비교 
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(20,4))

# Original images
for images, row in zip([images, output], axes):
    for img, ax in zip(images, row):
        ax.imshow(img.squeeze().numpy(), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

plt.show()

# 6. 잠재공간의 시각화 
- (3,3)으로 줄어든 잠재공간

In [ ]:

model.eval()  # Set the model to evaluation mode

latent_vectors = [] # 잠재공간 선언
labels = []

with torch.no_grad():
    for data in test_loader:
        imgs, lbls = data # 데이터 가져오기기
        imgs = imgs.view(imgs.size(0), -1) # flatten
        _, latents = model(imgs) # 이미지를 모델에 넣어서 잠재공간만 가져오기기
        latent_vectors.append(latents) # 차곡차곡 모으기기
        labels.append(lbls)

# 2. 잠재공간을 3d공간에서 표현할 수 있도록 시각화 하는 코드 
latent_vectors = torch.cat(latent_vectors, dim=0)
labels = torch.cat(labels, dim=0)

# 3D Visualization
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# 각각의 잠재공간 좌표들을 라벨과 색으로 표시
for i in range(10):
    indices = np.where(labels == i)[0]
    subset_indices = np.random.choice(indices, size=min(len(indices), 100), replace=False)

    for idx in subset_indices:
        ax.text(latent_vectors[idx, 0].item(), latent_vectors[idx, 1].item(), latent_vectors[idx, 2].item(),
                str(i), color=plt.cm.tab10(i), fontsize=9, ha='center')


# 좌표축 이름, 길이 설정
x_min, x_max = latent_vectors[:, 0].min(), latent_vectors[:, 0].max()
y_min, y_max = latent_vectors[:, 1].min(), latent_vectors[:, 1].max()
z_min, z_max = latent_vectors[:, 2].min(), latent_vectors[:, 2].max()

ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_zlim(z_min, z_max)


ax.set_xlabel('Latent X')
ax.set_ylabel('Latent Y')
ax.set_zlabel('Latent Z')

plt.show()